#### Actividad Modulo EDA y Regresión

In [73]:
##Importar librerias
##!pip install matplotlib
##!pip install seaborn 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [74]:
### Función cargar data sea .csv o .xlsx
def importDatos(ruta, archivo, ext, sepa):
    if ext == '.csv':
        data = pd.read_csv(ruta+archivo+ext, sep=sepa)
        return data
    elif ext == '.xlsx':
        data = pd.read_excel(ruta+archivo+ext)
        return data
    else:
        print('Extensión Diferente')

## Función para datos NAN -> Null -> 0 (IRQ) -> NAN
def datos_NAN(df):
    if isinstance(df, pd.DataFrame):
        total_na = df.isna().sum().sum()
        print("Dimesiones: %d filas, %d columnas" % (df.shape[0], df.shape[1]))
        print("Total de Valores NAN: %d" % (total_na))
        print("%40s %10s  %10s %10s" % ("Nombre Columna", "Tipo Dato", "#Unicos", "Cant Valores NAN"))
        col_name = df.columns
        dtyp = df.dtypes
        uniq = df.nunique()
        na_val = df.isna().sum()
        for i in range(len(df.columns)):
            print("%40s %10s  %10s %10s" % (col_name[i], dtyp[i], uniq[i], na_val[i]))
    else:
        print("Se esperaba un dataframe: %15s" % (type(df)))


In [75]:
### Cargue de datos
notas = importDatos('./Datasets/', 'student-mat','.csv', ';')

notas.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,10


In [76]:
datos_NAN(notas)

Dimesiones: 395 filas, 33 columnas
Total de Valores NAN: 0
                          Nombre Columna  Tipo Dato     #Unicos Cant Valores NAN
                                  school     object           2          0
                                     sex     object           2          0
                                     age      int64           8          0
                                 address     object           2          0
                                 famsize     object           2          0
                                 Pstatus     object           2          0
                                    Medu      int64           5          0
                                    Fedu      int64           5          0
                                    Mjob     object           5          0
                                    Fjob     object           5          0
                                  reason     object           4          0
                                gua

In [77]:
### identificamos las variables numéricas
numerico = [i for i in notas.columns if notas[i].dtype!='O']

numerico

['age',
 'Medu',
 'Fedu',
 'traveltime',
 'studytime',
 'failures',
 'famrel',
 'freetime',
 'goout',
 'Dalc',
 'Walc',
 'health',
 'absences',
 'G1',
 'G2',
 'G3']

In [78]:
### identificamos la variables categoricas que se van a trabajar
categoricas = [i for i in notas.columns if notas[i].dtype=='O']

categoricas

['school',
 'sex',
 'address',
 'famsize',
 'Pstatus',
 'Mjob',
 'Fjob',
 'reason',
 'guardian',
 'schoolsup',
 'famsup',
 'paid',
 'activities',
 'nursery',
 'higher',
 'internet',
 'romantic']

In [99]:
###Conversión de datos de tipo texto a numerico (variables dummies)
notas['school'] = camTextBi(notas,'school','GP',0,1)
notas['sex'] = camTextBi(notas,'sex','F',0,1)
notas['address'] = camTextBi(notas,'address','U',0,1)
notas['famsize'] = camTextBi(notas,'famsize','GT3',0,1)
notas['Pstatus'] = camTextBi(notas,'Pstatus','A',0,1)
notas['schoolsup'] = camTextBi(notas,'schoolsup','yes',0,1)
notas['famsup'] = camTextBi(notas,'famsup','yes',0,1)
notas['paid'] = camTextBi(notas,'paid','yes',0,1)
notas['activities'] = camTextBi(notas,'activities','yes',0,1)
notas['nursery'] = camTextBi(notas,'nursery','yes',0,1)
notas['higher'] = camTextBi(notas,'higher','yes',0,1)
notas['internet'] = camTextBi(notas,'internet','yes',0,1)
notas['romantic'] = camTextBi(notas,'romantic','yes',0,1)

datos_NAN(notas)


NameError: name 'camTextBi' is not defined

In [84]:
# agregar las columnas G1, G2 y G3
notas["final_grade"] = data[["G1", "G2", "G3"]].mean(axis=1)

# eliminar las columnas G1, G2 y G3 originales
data = notas.drop(["G1", "G2", "G3"], axis=1)

# mostrar las primeras 5 filas
data.head()

KeyError: "None of [Index(['G1', 'G2', 'G3'], dtype='object')] are in the [columns]"

In [79]:
notas.drop(['age','Medu','Fedu','traveltime','studytime','failures','famrel','freetime','goout','Dalc','Walc','health','absences'], axis=1, inplace=True)
notas.head()
#### Convertir cada variable categórica en variables numéricas de tipo dummy
##data= dummies = pd.get_dummies(notas, columns=categoricas)
#df = pd.get_dummies(notas, columns=['school', 'sex', 'address', 'famsize', 'Pstatus', 'Mjob', 'Fjob', 'reason', 'guardian','schoolsup','famsup','paid','activities','nursery','higher','internet','romantic'])
#df.head()

#datos_NAN(df)

,school,sex,address,famsize,Pstatus,Mjob,Fjob,reason,guardian,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,G1,G2,G3
0,GP,F,U,GT3,A,at_home,teacher,course,mother,yes,no,no,no,yes,yes,no,no,5,6,6
1,GP,F,U,GT3,T,at_home,other,course,father,no,yes,no,no,no,yes,yes,no,5,5,6
2,GP,F,U,LE3,T,at_home,other,other,mother,yes,no,yes,no,yes,yes,yes,no,7,8,10
3,GP,F,U,GT3,T,health,services,home,mother,no,yes,yes,yes,yes,yes,yes,yes,15,14,15
4,GP,F,U,GT3,T,other,other,home,father,no,yes,yes,no,yes,yes,no,no,6,10,10
